In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Lambda, Dense
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split

In [6]:
with open("CBOW.txt", "r") as file:
    text=file.read()

sentences= text.split('.')
sentences=[s.lower() for s in sentences]
sentences

['the speed of transmission is an important point of difference between the two viruses',
 ' influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than covid-19 virus',
 ' the serial interval for covid-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days',
 ' this means that influenza can spread faster than covid-19',
 '\n\nfurther, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission â€“ transmission of the virus before the appearance of symptoms â€“ is a major driver of transmission for influenza',
 ' in contrast, while we are learning that there are people who can shed covid-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission',
 '\n\nthe reproductive number â€“ the number of secondary infections generated from one infected individu

In [10]:
tokenizer= Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences=tokenizer.texts_to_sequences(sentences)
sequences

[[1, 38, 2, 3, 9, 39, 40, 41, 2, 42, 14, 1, 43, 23],
 [4,
  44,
  11,
  24,
  45,
  46,
  47,
  1,
  15,
  25,
  48,
  10,
  26,
  2,
  27,
  12,
  11,
  24,
  16,
  17,
  1,
  15,
  14,
  49,
  50,
  18,
  5,
  6,
  7],
 [1,
  16,
  17,
  8,
  5,
  6,
  7,
  9,
  51,
  10,
  19,
  20,
  52,
  21,
  28,
  8,
  4,
  7,
  1,
  16,
  17,
  9,
  29,
  21],
 [30, 53, 31, 4, 32, 54, 55, 18, 5, 6],
 [56,
  3,
  33,
  1,
  57,
  29,
  20,
  21,
  2,
  58,
  59,
  60,
  61,
  62,
  3,
  13,
  3,
  2,
  1,
  7,
  63,
  1,
  26,
  2,
  27,
  13,
  9,
  11,
  34,
  35,
  2,
  3,
  8,
  4],
 [33,
  64,
  28,
  65,
  22,
  66,
  31,
  67,
  22,
  68,
  69,
  32,
  70,
  5,
  6,
  7,
  71,
  72,
  73,
  74,
  10,
  75,
  76,
  77,
  78,
  30,
  79,
  80,
  81,
  10,
  19,
  11,
  34,
  35,
  2,
  3],
 [1,
  82,
  36,
  13,
  1,
  36,
  2,
  83,
  84,
  85,
  25,
  86,
  87,
  88,
  13,
  9,
  89,
  10,
  19,
  14,
  37,
  12,
  37],
 [20, 8, 5, 6, 7, 90, 18, 8, 4],
 [91,
  92,
  8,
  93,
  5,
  6,
  

In [12]:
idx2word= tokenizer.index_word
word2idx= tokenizer.word_index
print(idx2word, "\n\n", word2idx)

{1: 'the', 2: 'of', 3: 'transmission', 4: 'influenza', 5: 'covid', 6: '19', 7: 'virus', 8: 'for', 9: 'is', 10: 'to', 11: 'a', 12: 'and', 13: 'â€“', 14: 'between', 15: 'time', 16: 'serial', 17: 'interval', 18: 'than', 19: 'be', 20: '5', 21: 'days', 22: 'are', 23: 'viruses', 24: 'shorter', 25: 'from', 26: 'appearance', 27: 'symptoms', 28: 'while', 29: '3', 30: 'this', 31: 'that', 32: 'can', 33: 'in', 34: 'major', 35: 'driver', 36: 'number', 37: '2', 38: 'speed', 39: 'an', 40: 'important', 41: 'point', 42: 'difference', 43: 'two', 44: 'has', 45: 'median', 46: 'incubation', 47: 'period', 48: 'infection', 49: 'successive', 50: 'cases', 51: 'estimated', 52: '6', 53: 'means', 54: 'spread', 55: 'faster', 56: 'further', 57: 'first', 58: 'illness', 59: 'or', 60: 'potentially', 61: 'pre', 62: 'symptomatic', 63: 'before', 64: 'contrast', 65: 'we', 66: 'learning', 67: 'there', 68: 'people', 69: 'who', 70: 'shed', 71: '24', 72: '48', 73: 'hours', 74: 'prior', 75: 'symptom', 76: 'onset', 77: 'at', 78

In [13]:
vocab_size= len(idx2word)+1
emb_size=100
context_size=2

targets=[]
contexts=[]

for sequence in sequences:
    for i in range(context_size, len(sequence)-context_size):
        target=sequence[i]
        context=[sequence[i-2], sequence[i-1], sequence[i+1], sequence[i+2]]
        targets.append(target)
        contexts.append(context)
print(targets, "\n")
print(contexts)

[2, 3, 9, 39, 40, 41, 2, 42, 14, 1, 11, 24, 45, 46, 47, 1, 15, 25, 48, 10, 26, 2, 27, 12, 11, 24, 16, 17, 1, 15, 14, 49, 50, 18, 5, 17, 8, 5, 6, 7, 9, 51, 10, 19, 20, 52, 21, 28, 8, 4, 7, 1, 16, 17, 9, 31, 4, 32, 54, 55, 18, 33, 1, 57, 29, 20, 21, 2, 58, 59, 60, 61, 62, 3, 13, 3, 2, 1, 7, 63, 1, 26, 2, 27, 13, 9, 11, 34, 35, 2, 3, 28, 65, 22, 66, 31, 67, 22, 68, 69, 32, 70, 5, 6, 7, 71, 72, 73, 74, 10, 75, 76, 77, 78, 30, 79, 80, 81, 10, 19, 11, 34, 35, 36, 13, 1, 36, 2, 83, 84, 85, 25, 86, 87, 88, 13, 9, 89, 10, 19, 14, 37, 5, 6, 7, 90, 18, 8, 93, 5, 6, 12, 4, 23, 22, 94, 95, 12, 15, 96, 97, 98, 99] 

[[1, 38, 3, 9], [38, 2, 9, 39], [2, 3, 39, 40], [3, 9, 40, 41], [9, 39, 41, 2], [39, 40, 2, 42], [40, 41, 42, 14], [41, 2, 14, 1], [2, 42, 1, 43], [42, 14, 43, 23], [4, 44, 24, 45], [44, 11, 45, 46], [11, 24, 46, 47], [24, 45, 47, 1], [45, 46, 1, 15], [46, 47, 15, 25], [47, 1, 25, 48], [1, 15, 48, 10], [15, 25, 10, 26], [25, 48, 26, 2], [48, 10, 2, 27], [10, 26, 27, 12], [26, 2, 12, 11],

In [18]:
x=np.array(contexts)
y=np.array(targets)

In [19]:
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.20, random_state=42)

In [20]:
model= Sequential([
    Embedding(input_dim= vocab_size, output_dim=emb_size, input_length=2*context_size),
    Lambda(lambda x: tf.reduce_mean(x, axis=1)),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history=model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))
model.summary()

Epoch 1/50

5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.0095 - loss: 4.6242 - val_accuracy: 0.0000e+00 - val_loss: 4.6222
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1300 - loss: 4.5992 - val_accuracy: 0.0000e+00 - val_loss: 4.6190
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1184 - loss: 4.5696 - val_accuracy: 0.0000e+00 - val_loss: 4.6140
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1135 - loss: 4.5251 - val_accuracy: 0.0000e+00 - val_loss: 4.6046
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1010 - loss: 4.4434 - val_accuracy: 0.0000e+00 - val_loss: 4.5941
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1180 - loss: 4.3079 - val_accuracy: 0.0000e+00 - val_loss: 4.6069
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0869 - loss: 4.1202 - val_accuracy: 0.0000e+00 - val_loss: 4.7039
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0613 - loss: 3.9673 - val_accura

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 4, 100)         │        10,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 102)            │        52,326 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 659,900 (2.52 MB)

 Trainable params: 219,966 (859.24 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 439,934 (1.68 MB)

In [26]:
test_word=[]
for idx in x_test[5]:
    test_word.append(idx2word.get(idx))

input_data= np.expand_dims(x_test[5], axis=0)
pred=model.predict(input_data)
pred= np.argmax(pred[0])
print("i/p=>", test_word, "\n")
print("o/p=>", idx2word.get(pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
i/p=> ['interval', 'the', 'between', 'successive'] 

o/p=> of
